In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb  9 12:01:18 2020

@author: colin
"""

import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [15]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
df = pd.read_csv('transcript_sent.csv')
#print(df.target_names.unique())
df.head()

,Unnamed: 0,Sent
0,0,"Thank you, Kate, and good morning, everyone, a..."
1,1,With me to review Kroger’s second quarter 2017...
2,2,"As you know, Kroger has competed in an ever-ch..."
3,3,"Our success over time boils down to one thing,..."
4,4,That focus allows us to deepen our connection ...


In [17]:
# Convert to list
data = df.Sent.values.tolist()

pprint(data[:1])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


['Thank you, Kate, and good morning, everyone, and thank you for joining us']
[['thank', 'you', 'kate', 'and', 'good', 'morning', 'everyone', 'and', 'thank', 'you', 'for', 'joining', 'us']]


In [18]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

['thank', 'you', 'kate', 'and', 'good_morning_everyone', 'and', 'thank', 'you', 'for', 'joining', 'us']


In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thank', 'thank', 'join']]


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2)]]


In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.288*"year" + 0.261*"quarter" + 0.070*"second" + 0.053*"end" + 0.051*"net" '
  '+ 0.038*"earning" + 0.033*"trend" + 0.024*"become" + 0.019*"expectation" + '
  '0.017*"ahead"'),
 (1,
  '0.166*"make" + 0.121*"thing" + 0.118*"would" + 0.082*"price" + '
  '0.069*"marketing" + 0.055*"program" + 0.048*"need" + 0.039*"quality" + '
  '0.030*"side" + 0.026*"effort"'),
 (2,
  '0.147*"category" + 0.114*"get" + 0.081*"talk" + 0.074*"key" + 0.063*"gain" '
  '+ 0.046*"week" + 0.046*"expand" + 0.044*"mention" + 0.044*"even" + '
  '0.036*"half"'),
 (3,
  '0.265*"say" + 0.145*"volume" + 0.138*"big" + 0.092*"team" + 0.049*"today" + '
  '0.047*"country" + 0.038*"deep" + 0.028*"leadership" + 0.018*"talent" + '
  '0.010*"remind"'),
 (4,
  '0.283*"business" + 0.278*"continue" + 0.145*"grow" + 0.058*"next" + '
  '0.045*"focus" + 0.024*"strategic" + 0.023*"engagement" + 0.017*"keep" + '
  '0.014*"reason" + 0.013*"generate"'),
 (5,
  '0.318*"also" + 0.198*"company" + 0.141*"store" + 0.111*"want" + '
 

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -17.37940428350486

Coherence Score:  0.3264259077722638


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.182154 -0.091499       1        1  9.789118
19     0.163407 -0.057226       2        1  8.487928
4      0.228078  0.361846       3        1  8.275545
15     0.202637 -0.257501       4        1  7.851995
2      0.053796 -0.004568       5        1  6.684404
1     -0.008507  0.001443       6        1  5.673788
8      0.007844  0.000108       7        1  5.531688
18     0.017586 -0.000725       8        1  5.428896
11     0.007962  0.000144       9        1  5.235498
12    -0.030227  0.002598      10        1  4.794157
13    -0.023627  0.002310      11        1  4.586586
9     -0.073336  0.004529      12        1  3.984204
10    -0.063900  0.004119      13        1  3.831036
3     -0.084833  0.004822      14        1  3.494886
17    -0.086373  0.004792      15        1  3.449301
5     -0.071498  0.004369      16        1  3.427099
6     -0.066451  0.004194      17        1  3.245987
7     -0.108793  0.005399      18        1  2.680764
16    -0.118047  0.005458      19        1  2.097849
14    -0.127873  0.005387      20        1  1.449267, topic_info=          Term         Freq        Total Category  logprob  loglift
42      growth  2136.000000  2136.000000  Default  30.0000  30.0000
211      brand  2152.000000  2152.000000  Default  29.0000  29.0000
15        year  2253.000000  2253.000000  Default  28.0000  28.0000
5      quarter  2047.000000  2047.000000  Default  27.0000  27.0000
124   business  1874.000000  1874.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
788  portfolio     0.075915     1.864717  Topic20  -9.6336   1.0329
937   momentum     0.075910     1.864040  Topic20  -9.6337   1.0332
481      think     0.075925     1.867434  Topic20  -9.6335   1.0315
434     really     0.075916     1.865315  Topic20  -9.6336   1.0326
725     margin     0.075912     1.864501  Topic20  -9.6336   1.0329

[910 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
122       8  0.975840     ability
110      13  0.987767  accelerate
309       7  0.981967     account
306      12  0.978994    addition
355      15  0.983080     address
...     ...       ...         ...
63       10  0.997433        well
255      19  0.977021         win
90       11  0.995891        work
390       6  0.996016       would
15        1  0.999324        year

[295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 20, 5, 16, 3, 2, 9, 19, 12, 13, 14, 10, 11, 4, 18, 6, 7, 8, 17, 15])